In [1]:
import os 
from tqdm.notebook import tqdm
from llama_index.core import VectorStoreIndex, SimpleKeywordTableIndex, Document
import pandas as pd
import pickle
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import swifter


os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
file_path = "psgs_w100.tsv"
data = pd.read_csv(file_path, sep="\t")

In [ ]:

def process_row(row):
    return Document(
        text=row['text'],
        metadata={"id": row['id'], "title": row['title']}
    )

data['document'] = data.swifter.apply(
    lambda row: process_row(row), axis=1
)

# Convert to list
documents = data['document'].tolist()

Pandas Apply:   0%|          | 0/21015324 [00:00<?, ?it/s]

In [3]:
with open("llama_index_docs.pkl", "wb") as f:
    pickle.dump(documents, f)

Processing rows:   0%|          | 0/21015324 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
embedder = HuggingFaceEmbedding(model_name="sentence-transformers/gtr-t5-xxl")

index = VectorStoreIndex.from_documents(
    documents=documents,
    show_progress=True,
    transformations=[embedder]  
)

KeyboardInterrupt: 

In [4]:
data

,id,text,title
0,1,"Aaron Aaron ( or ; ""Ahärôn"") is a prophet, hig...",Aaron
1,2,God at Sinai granted Aaron the priesthood for ...,Aaron
2,3,his rod turn into a snake. Then he stretched o...,Aaron
3,4,"however, Aaron and Hur remained below to look ...",Aaron
4,5,"Aaron and his sons to the priesthood, and arra...",Aaron
...,...,...,...
21015319,21015320,"Rouvikonas Rouvikonas (Rubicon, in Greek ""Ρουβ...",Rouvikonas
21015320,21015321,"deemed insignificant by the police, authoritie...",Rouvikonas
21015321,21015322,Limelight centre Limelight is a community heal...,Limelight centre
21015322,21015323,best older people's housing development and Be...,Limelight centre
